In [1]:
import numpy as np
import pandas as pd
import re
import nltk

In [33]:
def clean_str(s):
    s = re.sub(r"http\S+", "", s)
    s = re.sub(r"#(\w+)", ' ', s, flags=re.MULTILINE) 
    s = re.sub(r"@(\w+)", ' ', s, flags=re.MULTILINE)
    s = re.sub(r"&amp", " ", s)
    s = re.sub(r"[^A-Za-z0-9:(),!?\'\`]", " ", s)
    s = re.sub("\d+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\'s", " ", s)
    s = re.sub(r"\'m", " am", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'d", " ", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\(", " ", s)
    s = re.sub(r"\)", " ", s)
    return s

In [2]:
owndata = pd.read_csv('/Users/lincong/Downloads/CS-226-Twitter-Bitcoin-Tweet-Sentimental-Analysis/new_preprocessed_tweets.csv')

In [4]:
owndata['timestamp'].max()

'2019-05-27 12:21:50+00'

In [5]:
owndata['timestamp'].min()

'2009-01-11 03:33:52+00'

In [24]:
owndata.drop(['Unnamed: 0', 'text'], axis = 1, inplace = True)

In [43]:
#further clean the tweet text
texts = []

for idx in range(owndata.clean_text.shape[0]):
    text = owndata.clean_text[idx]
    cleaned_str = clean_str(text)
    cleaned_str = ' '.join(cleaned_str.split())
    texts.append(cleaned_str)

In [49]:
texts_df = pd.DataFrame(texts, columns=['cleaned_text'])
cleaned_data = pd.concat([owndata, texts_df], axis = 1).drop(['clean_text'], axis = 1)
cleaned_data['timestamp'] = pd.to_datetime(cleaned_data['timestamp']).dt.date

In [70]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [71]:
words = set(nltk.corpus.words.words())

In [77]:
# keep only english words
texts_eng = []

for idx in range(cleaned_data.cleaned_text.shape[0]):
    text = cleaned_data.cleaned_text[idx]
    cleaned_str = " ".join(w for w in nltk.wordpunct_tokenize(text) \
                           if w.lower() in words or not w.isalpha())
    texts_eng.append(cleaned_str)

In [79]:
texts_eng_df = pd.DataFrame(texts_eng, columns=['cleaned_eng_text'])
cleaned_eng_data = pd.concat([cleaned_data, texts_eng_df], axis = 1).drop(['cleaned_text'], axis = 1)

In [7]:
# collect the bitcoin previous data 
btcdata = pd.read_csv('/Users/lincong/Downloads/coin_Bitcoin.csv')
btcdata['Date'] = pd.to_datetime(btcdata['Date'])  
mask = (btcdata['Date'] > '2009-1-10') & (btcdata['Date'] <= '2019-5-28')
btc_to_combine = btcdata.loc[mask]

In [18]:
btc_to_combine.drop(['SNo', 'Name', 'Symbol'], axis = 1, inplace = True)
btc_to_combine['Date'] = pd.to_datetime(btc_to_combine['Date']).dt.date

/Users/lincong/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [118]:
# shift to get the previous day features
one_time_lag = btc_to_combine.shift(1)

In [119]:
one_time_lag.columns = one_time_lag.columns + '_pre'

In [123]:
btc_to_combine_final = pd.concat([btc_to_combine, one_time_lag], axis = 1).dropna().drop('Date_pre', axis = 1)

In [124]:
# combine bitcoin preice data with the tweet text
combined_data = pd.merge(cleaned_eng_data, btc_to_combine_final, how="left", left_on='timestamp', right_on = 'Date')
combined_data.drop(['Date'], axis = 1, inplace = True)

In [126]:
# check the missingness
combined_data.isna().sum()

timestamp            0
textblob_score       0
textblob_category    0
vader_score          0
vader_category       0
cleaned_eng_text     0
High                 2
Low                  2
Open                 2
Close                2
Volume               2
Marketcap            2
High_pre             2
Low_pre              2
Open_pre             2
Close_pre            2
Volume_pre           2
Marketcap_pre        2
dtype: int64

In [129]:
combined_data['cleaned_eng_text'].replace('', np.nan, inplace=True)

In [130]:
complete_data = combined_data.dropna()

In [131]:
complete_data = complete_data.reset_index()

In [133]:
complete_data.drop('index', axis=1, inplace = True)

In [136]:
# use only textblob score and original tweet text for modeling
textblob_data = complete_data.drop(['vader_score', 'vader_category'], axis = 1)

In [138]:
textblob_data.to_csv('/Users/lincong/Downloads/textblob_data.csv')